In [1]:
import os
os.chdir("/app")
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from torchinfo import summary

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# El modelo que se quiere afinar desde el hub de Hugging Face
# Se seleccionan dos grandes modelos, ambos de alrededor de 7B-8B parámetros, lo que afecta directamente al uso de VRAM.

model_name = "Qwen/Qwen2.5-Coder-3B-Instruct"

# El dataset de instrucciones a utilizar para el ajuste fino
# Afecta a la tarea que el modelo realizará, en este caso, instrucciones.
local_dataset_path = "angular_dataset.csv"

# Nombre del modelo ajustado
new_model = f"{model_name}-angular-trained"

model_layers = ['q_proj','k_proj','v_proj','o_proj']

################################################################################
# Parámetros de QLoRA (Fine-tuning con LoRA de bajo rango)
################################################################################
# LoRA (Adaptación de Bajo Rango) es una técnica utilizada para reducir el número de parámetros que necesitan ser ajustados durante el fine-tuning.
# En lugar de actualizar todos los parámetros del modelo, LoRA inserta capas de bajo rango en matrices de proyección de atención. 
# Esto permite un ajuste eficiente con menor uso de VRAM, ya que solo se entrenan estas capas adicionales.
# En términos de memoria y tiempo de cómputo, manteniendo gran parte de los parámetros originales congelados.

# Dimensión de la atención en LoRA
# Determina el tamaño del cuello de botella en LoRA; valores más altos requieren más VRAM pero permiten capturar más información.
lora_r = 32

# Parámetro Alpha para el escalado de LoRA
# Afecta la intensidad del ajuste fino aplicado; valores más altos pueden mejorar el rendimiento pero aumentan el uso de memoria.
lora_alpha = 16

# Probabilidad de Dropout en las capas LoRA
# Dropout introduce regularización, reduciendo el sobreajuste y el uso de VRAM en la activación de capas.
lora_dropout = 0.1

################################################################################
# Parámetros de bitsandbytes (Quantización)
################################################################################
# La cuantización es una técnica utilizada para reducir el tamaño de los modelos y el uso de VRAM al representar los pesos del modelo con menor precisión, 
# por ejemplo, usando 4 bits en lugar de 16 o 32. 
# Esto permite cargar modelos más grandes en GPUs con menos memoria. 
# Aunque puede introducir una leve pérdida de precisión, la cuantización permite hacer más eficiente el entrenamiento y la inferencia.
# En esta configuración se utiliza la cuantización de 4 bits (opcional), y se pueden usar diferentes tipos como fp4 y nf4, que ofrecen un buen 
# balance entre eficiencia y rendimiento.

# Activar carga de modelos base con precisión de 4 bits - Necesita GPU
# Reduce el uso de VRAM y permite entrenar modelos más grandes en GPUs con menos memoria.
use_4bit = False

# Tipo de datos de cómputo para modelos base de 4 bits
# El uso de float16 reduce el uso de VRAM en comparación con float32.
bnb_4bit_compute_dtype = "float16"

# Tipo de cuantización (fp4 o nf4)
# nf4 es más eficiente en términos de memoria y precisión en comparación con fp4.
bnb_4bit_quant_type = "nf4"

# Activar cuantización anidada para modelos base de 4 bits (cuantización doble)
# Aumenta la compresión, reduciendo aún más la VRAM utilizada, pero puede afectar el rendimiento.
use_nested_quant = False

################################################################################
# Parámetros de TrainingArguments (Ajustes de entrenamiento)
################################################################################

# Directorio de salida donde se almacenarán las predicciones y checkpoints del modelo
output_dir = "./results"

# Número de épocas de entrenamiento
# Afecta la duración del entrenamiento; más épocas pueden mejorar la generalización pero requieren más tiempo.
num_train_epochs = 1

# Activar entrenamiento con fp16/bf16 (usar bf16 si se tiene una GPU A100/H100)
# fp16/bf16 reduce significativamente el uso de VRAM, lo que permite entrenar modelos más grandes.
fp16 = False
bf16 = True

# Tamaño del batch por GPU para entrenamiento y evaluación
# Un mayor tamaño de batch incrementa el uso de VRAM, pero puede acelerar el entrenamiento si la memoria lo permite.
per_device_train_batch_size = 4
per_device_eval_batch_size = 4

# Número de pasos de actualización para acumular los gradientes
# Aumenta el uso de memoria temporalmente pero permite actualizar los gradientes menos frecuentemente.
gradient_accumulation_steps = 1

# Activar checkpointing de gradientes
# Reduce el uso de VRAM al recalcular activaciones durante el retropropagado, a cambio de un mayor tiempo de cómputo.
gradient_checkpointing = True

# Máxima norma del gradiente (clipping)
# Previene explosiones de gradiente, especialmente útil en grandes modelos, pero afecta el uso de memoria.
max_grad_norm = 0.3

# Tasa de aprendizaje inicial (optimizador AdamW)
# Controla la velocidad de convergencia, un valor más alto puede requerir mayor memoria debido a la intensidad de los ajustes.
learning_rate = 2e-4

# Decaimiento del peso para todas las capas excepto bias/pesos de LayerNorm
# Afecta la regularización durante el entrenamiento, influyendo en cómo se ajustan los pesos del modelo.
weight_decay = 0.001

# Optimizador a usar
# En caso de utilizar un modelo de 4 bits, se utiliza paged_adamw_32bit, optimizado para la eficiencia de memoria.
optim = "paged_adamw_32bit" if use_4bit else "adamw_torch"

# Programador de tasa de aprendizaje
# El programador "cosine" ajusta la tasa de aprendizaje de forma gradual, lo que puede afectar el tiempo de convergencia y el uso de memoria.
lr_scheduler_type = "cosine"

# Número máximo de pasos de entrenamiento (anula num_train_epochs si se establece)
# Controla cuántos pasos de entrenamiento ejecutar; valores mayores requieren más tiempo y uso de VRAM.
max_steps = -1

# Proporción de pasos para un calentamiento lineal (de 0 a la tasa de aprendizaje)
# Un mayor warmup suaviza el inicio del entrenamiento, lo que puede ayudar a estabilizar los gradientes en grandes modelos.
warmup_ratio = 0.03

# Agrupar secuencias en batches con la misma longitud
# Ahorra memoria y acelera el entrenamiento al minimizar el relleno innecesario en secuencias más cortas.
group_by_length = True

# Guardar un checkpoint cada X pasos de actualización
# Afecta la frecuencia con la que se almacenan los modelos; más checkpoints requieren más espacio de almacenamiento pero permiten recuperar más fácilmente.
save_steps = 0

# Registrar logs cada X pasos de actualización
# Afecta la frecuencia de logging, sin impacto directo en la VRAM pero sí en el rendimiento.
logging_steps = 50

################################################################################
# Parámetros de SFT (Supervised Fine-Tuning)
################################################################################

# Longitud máxima de secuencia
# Limita la longitud de las secuencias procesadas; secuencias más largas usan más VRAM.
max_seq_length = 1024

# Empaquetar múltiples ejemplos cortos en la misma secuencia de entrada para aumentar la eficiencia
# Mejora la eficiencia del entrenamiento, agrupando ejemplos cortos para reducir el relleno y ahorrar VRAM.
packing = True

# Cargar el modelo entero en la GPU 0
# Especifica en qué dispositivo se carga el modelo; si es GPU, afectará el uso de VRAM.
device_map = {"": 0} if torch.cuda.is_available() else {"": "cpu"}

In [3]:
# Cargar el dataset (puedes procesarlo aquí)
# El dataset se carga desde Hugging Face, usando el nombre especificado en "dataset_name".
dataset = load_dataset('csv', data_files=local_dataset_path, split='train')
print(dataset)
print(dataset['text'][0])

# Cargar el tokenizer y el modelo con la configuración de QLoRA
# Se selecciona el tipo de datos de cómputo (float16 en este caso) para el modelo.
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Si se activa el uso de 4 bits, se configura la cuantización BitsAndBytes (para reducir el uso de VRAM).
if use_4bit:
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=use_4bit,
        bnb_4bit_quant_type=bnb_4bit_quant_type,
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=use_nested_quant,
    )
else:
    quantization_config = None

# Verifica la compatibilidad de la GPU con bfloat16 (mejora en entrenamiento y uso de memoria)
if torch.cuda.is_available() and compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)


# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map=device_map
)
# Desactiva el uso de cache para reducir la memoria durante el entrenamiento
model.config.use_cache = False 
# Controla el particionamiento en modelos grandes para paralelización
model.config.pretraining_tp = 1  

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Cargar la configuración de LoRA (Low-Rank Adaptation)
# LoRA permite ajustar solo ciertas partes del modelo (proyectores de atención) para ahorrar memoria.
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=model_layers
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Calculate total and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
percentage_trainable = (trainable_params / total_params) * 100

# Print a detailed summary using torchinfo
if not use_4bit:
    print(summary(model, input_size=(1, 512), dtypes=[torch.int]))

# Print a summary of the model
print("Model Summary:")
print(f"Model name: {model_name}")
print(f"Number of parameters: {total_params / 1e9:.2f} billion")
print(f"Number of trainable parameters: {trainable_params / 1e9:.2f} billion")
print(f"Percentage of trainable parameters: {percentage_trainable:.2f}%")
print(f"Number of layers: {model.config.num_hidden_layers}")
print(f"Hidden size: {model.config.hidden_size}")

print('=========================================================================================================')

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")

    device = torch.device("cuda")
    current_gpu_name = torch.cuda.get_device_name(device.index)
    print(f"Using GPU: {current_gpu_name}")

    memory_stats = torch.cuda.memory_stats(device)
    total_memory = torch.cuda.get_device_properties(device).total_memory
    allocated_memory = memory_stats['allocated_bytes.all.current']
    reserved_memory = memory_stats['reserved_bytes.all.current']
    free_memory = total_memory - reserved_memory

    print(f"Total VRAM: {total_memory / (1024 ** 3):.2f} GB")
else:
    device = torch.device("cpu")
    print("Using CPU")

print('=========================================================================================================')

Generating train split: 220 examples [00:00, 7273.97 examples/s]


Dataset({
    features: ['text'],
    num_rows: 220
})
guide accessibility
# Accessibility in Angular

The web is used by a wide variety of people, including those who have visual or motor impairments. A variety of assistive technologies are available that make it much easier for these groups to interact with web-based software applications. Also, designing an application to be more accessible generally improves the user experience for all users.

For an in-depth introduction to issues and techniques for designing accessible applications, see the Accessibility section of the Google's Web Fundamentals.

This page discusses best practices for designing Angular applications that work well for all users, including those who rely on assistive technologies.

> For the sample application that this page describes, see the live example.

## Accessibility attributes

Building accessible web experience often involves setting Accessible Rich Internet Applications (ARIA) attributes to provide seman

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.66s/it]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, packing. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:212: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `datase

Layer (type:depth-idx)                             Output Shape              Param #
Qwen2ForCausalLM                                   [1, 512, 151936]          --
├─Qwen2Model: 1-1                                  [1, 512, 2048]            --
│    └─Embedding: 2-1                              [1, 512, 2048]            (311,164,928)
│    └─Qwen2RotaryEmbedding: 2-2                   [1, 512, 128]             --
│    └─ModuleList: 2-3                             --                        --
│    │    └─Qwen2DecoderLayer: 3-1                 [1, 512, 2048]            77,486,592
│    │    └─Qwen2DecoderLayer: 3-2                 [1, 512, 2048]            77,486,592
│    │    └─Qwen2DecoderLayer: 3-3                 [1, 512, 2048]            77,486,592
│    │    └─Qwen2DecoderLayer: 3-4                 [1, 512, 2048]            77,486,592
│    │    └─Qwen2DecoderLayer: 3-5                 [1, 512, 2048]            77,486,592
│    │    └─Qwen2DecoderLayer: 3-6                 [1, 512, 2048

In [4]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "To create an Angular service named `AuthService` that contains a method"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

To create an Angular service named `AuthService` that contains a method for logging in users, you can follow these steps:

1. **Create the AuthService Class**:
   - Open your Angular project and navigate to the `src/app` directory.
   - Create a new file named `auth.service.ts`.
   - Add the following code to define the `AuthService` class:

```typescript
import { Injectable } from '@angular/core';

@Injectable({
  providedIn: 'root'
})
export class AuthService {
  private isLoggedIn = false;

  // Method to simulate user login
  login(username: string, password: string): boolean {
    // For demonstration purposes, let's assume the username is "admin" and password is "password"
    if (username === "admin" && password === "password") {
      this.isLoggedIn = true;
      return true;
    }
    return false;
  }

  // Method to check if the user is logged in
  isLoggedIn(): boolean {
    return this


In [5]:
# Train model
trainer.train()

Step,Training Loss
50,1.629800
100,1.526400


TrainOutput(global_step=122, training_loss=1.5708340191450276, metrics={'train_runtime': 61.5261, 'train_samples_per_second': 7.915, 'train_steps_per_second': 1.983, 'total_flos': 8346598983598080.0, 'train_loss': 1.5708340191450276, 'epoch': 1.0})

In [6]:
trainer.model.save_pretrained(new_model)
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.save_pretrained(new_model)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.60s/it]


('Qwen/Qwen2.5-Coder-3B-Instruct-angular-trained/tokenizer_config.json',
 'Qwen/Qwen2.5-Coder-3B-Instruct-angular-trained/special_tokens_map.json',
 'Qwen/Qwen2.5-Coder-3B-Instruct-angular-trained/vocab.json',
 'Qwen/Qwen2.5-Coder-3B-Instruct-angular-trained/merges.txt',
 'Qwen/Qwen2.5-Coder-3B-Instruct-angular-trained/added_tokens.json',
 'Qwen/Qwen2.5-Coder-3B-Instruct-angular-trained/tokenizer.json')

In [7]:
# Run text generation pipeline with our next model
pipe = pipeline(task="text-generation", model=new_model, tokenizer=tokenizer, max_length=200)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.13s/it]


To create an Angular service named `AuthService` that contains a method to authenticate users, you can follow these steps:

1. **Install Angular CLI**: If you haven't already, install the Angular CLI globally on your machine by running:
   ```bash
   npm install -g @angular/cli
   ```

2. **Create a New Angular Project**: If you don't have an existing project, create a new one using the Angular CLI:
   ```bash
   ng new my-angular-app
   cd my-angular-app
   ```

3. **Generate a Service**: Use the Angular CLI to generate a new service named `AuthService`:
   ```bash
   ng generate service auth
   ```

4. **Implement the AuthService**: Open the `auth.service.ts` file and implement the authentication logic. Here's a simple example using HTTP requests:

   ```typescript
   import { Injectable } from '@angular/core';
   import { HttpClient } from '@angular/common/http';
  
